<a href="https://colab.research.google.com/github/MSaydurRahman/image-processing-with-kaggle-data-set/blob/main/image_processing_with_kaggle_data_set_using_VGG19_part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import pathlib
from torch.utils.data import DataLoader
from torchvision import *

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
transformtrain = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])
transformtest = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainds = datasets.ImageFolder('/content/drive/My Drive/Data/seg_train/seg_train', transform=transformtrain)
testds = datasets.ImageFolder('/content/drive/My Drive/Data/seg_test/seg_test', transform=transformtest)

In [ ]:
trainloader = DataLoader(trainds, batch_size=256, shuffle=True)
testloader = DataLoader(testds, batch_size=64, shuffle=False)

In [ ]:
root = pathlib.Path('/content/drive/My Drive/Data/seg_train/seg_train')
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
model = models.vgg19(pretrained=True).to(device)
for param in model.features.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


In [ ]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
 model.classifier = nn.Sequential(
      
  nn.Linear(in_features=25088, out_features=2048) ,
  nn.ReLU(),
  nn.Linear(in_features=2048, out_features=512) ,
  nn.ReLU(),
  nn.Dropout(p=0.6), 
    
  nn.Linear(in_features=512 , out_features=6),
  nn.LogSoftmax(dim=1)  
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.00001)

In [ ]:
trainlosses = []
testlosses = []
for e in range(50):
    trainloss = 0
    traintotal = 0
    trainsuccessful = 0
    for traininput, trainlabel in trainloader:
        traininputs, trainlabels = traininput.to(device), trainlabel.to(device)
        optimizer.zero_grad()
        trainpredictions = model(traininputs)
        _, trainpredict = torch.max(trainpredictions.data, 1)
        loss = criterion(trainpredictions, trainlabels)
        loss.backward()
        optimizer.step()
        trainloss += loss.item()
        traintotal += trainlabels.size(0)
        trainsuccessful += (trainpredict == trainlabels).sum().item()
    else:
        testloss = 0
        testtotal = 0
        testsuccessful = 0
        with torch.no_grad():
            for testinput, testlabel in testloader:
                testinputs, testlabels = testinput.to(device), testlabel.to(device)
                testpredictions = model(testinputs)
                _, testpredict = torch.max(testpredictions.data, 1)
                tloss = criterion(testpredictions, testlabels)
                testloss += tloss.item()
                testtotal += testlabels.size(0)
                testsuccessful += (testpredict == testlabels).sum().item()
        trainlosses.append(trainloss/len(trainloader))
        testlosses.append(testloss/len(testloader))
        print('Train Accuracy %{:.2f}'.format(100*trainsuccessful/traintotal))
        print('Test Accuracy %{:.2f}'.format(100*testsuccessful/testtotal))

Train Accuracy %65.95
Test Accuracy %83.57
Train Accuracy %85.87
Test Accuracy %86.83
Train Accuracy %88.29
Test Accuracy %88.73
Train Accuracy %89.55
Test Accuracy %89.53
Train Accuracy %90.20
Test Accuracy %89.67
Train Accuracy %90.75
Test Accuracy %89.97
Train Accuracy %91.24
Test Accuracy %89.80
Train Accuracy %91.47
Test Accuracy %90.27
Train Accuracy %91.96
Test Accuracy %90.67
Train Accuracy %92.50
Test Accuracy %90.87
Train Accuracy %92.77
Test Accuracy %91.33
Train Accuracy %92.80
Test Accuracy %90.60
Train Accuracy %93.24
Test Accuracy %91.20
Train Accuracy %93.61
Test Accuracy %91.13
Train Accuracy %94.03
Test Accuracy %91.50
Train Accuracy %93.99
Test Accuracy %91.13
Train Accuracy %94.38
Test Accuracy %91.10
Train Accuracy %94.33
Test Accuracy %91.73
Train Accuracy %94.66
Test Accuracy %91.23
Train Accuracy %95.05
Test Accuracy %91.53
Train Accuracy %95.10
Test Accuracy %91.27
Train Accuracy %95.25
Test Accuracy %91.53
Train Accuracy %95.61
Test Accuracy %91.07
Train Accur